In [1]:
%run ~/Initialize_tianyu.ipynb

In [3]:
import hax
print(hax.__version__)
hax.init(experiment='XENON1T',
         pax_version_policy='loose',
         sc_api_key='ssn1sslkvdhittoywjk88w9cmbxf4fmprn68r8hm',
         force_reload=False,
         raw_data_access_mode = 'local',
         raw_data_local_path = ['/project/lgrandi/xenon1t/raw_for_waveforms/'], 
         main_data_paths= [
             '/project2/lgrandi/xenon1t/processed/pax_v6.10.0',
             '/project/lgrandi/xenon1t/processed/pax_v6.10.1'],
         minitree_paths=[
             '/project/lgrandi/xenon1t/minitrees/pax_v6.10.0', # 
             '/project/lgrandi/xenon1t/minitrees/pax_v6.10.1', #
             '/project/lgrandi/feigao/minitrees/pax_v6.10.0', # Fei's new positions
             '/project2/lgrandi/xenon1t/minitrees/pax_v6.10.0', # Reprocessed data are here
            ])

2.5.0


In [12]:
import glob
def find_downloaded_events(df):
    global dsets
    dsets = hax.runs.datasets
    df['raw_data']=False
    for run_number in df.run_number.unique():
        run_name = dsets.loc[dsets.number==run_number, 'name'].values[0]
        for folder in glob.glob(hax.config['raw_data_local_path'][0]+'/*%s*'%run_name):
            for file in os.listdir(folder):
                min_event_number, max_event_number = file.split('-')[2:4]
                df.loc[(df.run_number==run_number) 
                       & (df.event_number>int(min_event_number))
                       & (df.event_number<int(max_event_number)),
                       'raw_data'
                      ] = True
    return df

In [37]:
d = pd.read_pickle('wf_samples.pkl')
d = find_downloaded_events(d)
d = d[d.raw_data]

In [38]:
d.sort_values(by=['run_number', 'event_number'], inplace=True)
run_number = d.run_number.unique()[0]
tmp = d[d.run_number==run_number]

In [39]:
from pax import units
from pax import configuration

config_names=('_base', 'XENON1T')
c = configuration.load_configuration(config_names)

event_numbers = tmp.event_number.unique()
run_name = dsets.loc[dsets.number==run_number, 'name'].values[0]

from pax import core
core_processor = core.Processor(
    config_names=('_base','XENON1T'),
    just_testing=False,
    config_dict={
        'pax':{
            'input_name' : '/project/lgrandi/xenon1t/raw_for_waveforms/%s'%run_name,
            'events_to_process' : event_numbers,
            'output' : 'Dummy.DummyOutput',
            'pre_output' : [],
            'encoder_plugin' : None,
            }}
)
events = core_processor.get_events()

processor MainProcess L66 INFO This is PAX version 6.10.1, running with configuration for XENON1T.
ReadZipped MainProcess L102 INFO InputFromFolder: Selecting file /project/lgrandi/xenon1t/raw_for_waveforms/170209_2348/XENON1T-6969-000004000-000004999-000001000.zip (number 1/3 in folder) for reading


In [33]:
if not '/home/zhut/Beta_Beta/Calibration_MS/' in sys.path:
    sys.path.append('/home/zhut/Beta_Beta/Calibration_MS/')
from MultipleS2Peaks_PUpdate import MultipleS2Peaks
ms2p = MultipleS2Peaks()

def classify(peak, s1, s2):
    if peak.type == 's1':
        return peak.type
    if peak.type == 's2':
        if peak.area<150:
            return 'single_e'
        drift_time = (peak.index_of_maximum - s1.index_of_maximum) * ms2p.sample_duration
        z = - ms2p.drift_velocity_liquid * (drift_time -ms2p.drift_time_gate)
        for rp in peak.reconstructed_positions:
            if rp.algorithm == 'PosRecTopPatternFit':
                gof = getattr(rp, 'goodness_of_fit')
        ans = ms2p.determine_interaction(pd.DataFrame([dict(z=z, 
                                                     area=peak.area, 
                                                     goodness_of_fit_tpf=gof,
                                                     range_50p_area=list(peak.range_area_decile)[5],
                                                     s2=s2.area
                                                    )]))
        if not ans.not_interaction.values[0]:
            return 's2'
        else:
            return 'e_train'

In [16]:
############## Plotting while processing #######################
def hit_color(hits):
    color_factor = np.clip(hits.height/hits.noise_sigma, 0, 15)/15
    is_rejected = hits.is_rejected.astype(int)
    
    aplist = c['DesaturatePulses.DesaturatePulses']['large_after_pulsing_channels']
    offlist = list(np.where(np.array(c['DEFAULT']['gains'])==0.0)[0])
    
    is_ap = hits.channel.isin(aplist).astype(int)
    is_off = hits.channel.isin(offlist).astype(int)
    is_any = 1- (1-is_ap) * (1-is_off) * (1-is_rejected)
    
    rgba_colors = np.zeros((len(hits), 4))
    rgba_colors[:, 0] = (1 - is_any) * color_factor
    rgba_colors[:, 1] = is_off
    rgba_colors[:, 2] = is_ap #(1 - is_rejected) * (1 - color_factor)
    rgba_colors[:, 3] = 0.75
    
    return rgba_colors

def plot_2d_waveform(event, xlim):

    global investigating_peaks
    investigating_peaks = []
    
    if len(event.interactions) < 1:
        print('No interaction in this event')
        return 0

    s1 = event.peaks[event.interactions[0].s1]
    s2 = event.peaks[event.interactions[0].s2]
        
    with initiate_plot(30,15):
        axm = plt.gca()
        axm.tick_params(axis='both', bottom='off', labelbottom='off', left='off', labelleft='off')
        pos = axm.get_position()
        top = [pos.x0, pos.y0+0.5*pos.height, pos.width, pos.height*0.5]
        bot = [pos.x0, pos.y0, pos.width, pos.height*0.5]

        ####################################################
        axt = plt.axes(top)
        w = event.get_sum_waveform('tpc').samples[:]
        time = np.arange(len(w))*0.01

        ymax = np.max(w)**1.5

        plt.yscale('symlog')
        plt.plot(time, w, color='k', lw=2.0, zorder=1)
    
        plt_config(xlim = xlim, ylim=[-1, ymax], ylabel='Amplitude [pe/bin]')
        axt.tick_params(axis='x', bottom='off', labelbottom='off', left='off', labelleft='off')

        ####################################################
        axb = plt.axes(bot)
        plt.axhline(127, color='k', zorder=0)
        for peak_i, peak in enumerate(event.peaks):
            if peak.detector != 'tpc': continue
            if peak.right/100 < xlim[0] or peak.left/100 > xlim[1]: continue

            #global hits
            hits = pd.DataFrame(peak.hits)
            hits = hits[(hits.channel<248) & (hits.area>2)]            
            axb.scatter(hits.index_of_maximum*0.01, hits.channel, 
                        c=hit_color(hits), edgecolor='none', s=10 * np.clip(hits.area, 0, 10))

            if (peak.area>100) and (peak.type == 's1' or peak.type == 's2'):
                x, y = peak.index_of_maximum*0.01, w[peak.index_of_maximum]
                ytext = np.random.uniform(y, min(ymax, y*5))

                if x>xlim[1] or x<xlim[0]:
                    continue
                axt.axvspan(peak.left*0.01, (peak.right-1)*0.01, color='grey', alpha=0.1)
                axb.axvspan(peak.left*0.01, (peak.right-1)*0.01, color='grey', alpha=0.1)
                axt.text(x, ytext, text)

                cmap = dict(s1='C0', s2='C1', e_train='C2', single_e='C4')
                axt.scatter(x, y, color=cmap[text])

                investigating_peaks.append(dict(type=text, peak=peak, hits=hits))

        plt_config(xlim=xlim, ylim=[0, 249], xlabel='Time [$\mu s$]', ylabel='PMT channel')

In [ ]:
for event_i in range(10):
    event_pax_core = next(events)
    event_pax_core = core_processor.process_event(event_pax_core)
    sec = tmp[tmp.event_number==event_pax_core.event_number]
    sec = sec[~sec.not_interaction]
    if len(sec) > 1:
        s1ct = event_pax_core.peaks[event_pax_core.interactions[0].s1].center_time

        left = min(s1ct, sec.center_time.min())/1000 - 10
        right = sec.center_time.max()/1000 + 10
        plot_2d_waveform(event_pax_core, [left, right])
        fig.savefig('waveforms/run%d_event%d.png'%(run_number, event_pax_core.event_number), dpi=fig.dpi)
